In [ ]:
import random
import pandas as pd
import numpy as np

In [ ]:
E = [(1,2),(1,9),(1,14),(2,3),(2,5),(3,4),(3,5),(3,6),(3,7),(4,6),(5,7),(5,9),(5,10),(6,7),(6,8),(7,8),(7,11),(8,11),(9,10),(9,13),(9,14),(10,11),(10,12),(12,16),(13,15),(13,16),(14,15),(15,16),
    (2,1),(9,1),(14,1),(3,2),(5,2),(4,3),(5,3),(6,3),(7,3),(6,4),(7,5),(9,5),(10,5),(7,6),(8,6),(8,7),(11,7),(11,8),(10,9),(13,9),(14,9),(11,10),(12,10),(16,12),(15,13),(16,13),(15,14),(16,15)]
df = np.zeros((16, 16))
for row, col in E:
    df[row-1, col-1] = 1
df = pd.DataFrame(df, columns=range(1, 17), index=range(1, 17))
df

In [134]:
class Agent():
    def __init__(self, df, num_genes=16):
        self.num_genes = num_genes
        self.chromosome = [bool(random.choice([0, 1])) for _ in range(num_genes)]
        self.fitness = self.calc_fitness()
        self.create_groups()
        self.connections = 0
        self.reward = self.calc_reward()
    
    def create_groups(self):
        group1 = [i for i in range(self.num_genes) if self.chromosome[i] == 1]
        group2 = [i for i in range(self.num_genes) if self.chromosome[i] == 0]
        self.group1 = group1
        self.group2 = group2
    
    def calc_connections(self, group1, group2):
        inter_group_edges = [edge for edge in E if (edge[0] in group1 and edge[1] in group2) or (edge[0] in group2 and edge[1] in group1)]
        print(inter_group_edges)
        return len(inter_group_edges)
    
    def check_factible(self):
        ...
    def calc_reward(self):
        ...
    
    def mutate(self):
        ...

    def calc_fitness(self):
        ...

In [135]:
E = [(1,2),(1,9),(1,14),(2,3),(2,5),[3,4],[3,5],
    [3,6],[3,7],[4,6],[5,7],[5,9],[5,10],[6,7],
    [6,8],[7,8],[7,11],[18,11],[9,10],[9,13], [9,14], [10,11],[10,12],[12,16],[13,15],[13,16],
    [14,15],[15,16]]

yo = Agent(df)


In [140]:
yo.group1, yo.group2

([0, 1, 3, 5, 7, 8, 9, 13, 14], [2, 4, 6, 10, 11, 12, 15])

In [130]:
yo.group1

[8, 9, 10, 11, 12, 13, 14, 15]

In [106]:
yo.chromosome = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1]
yo.create_groups()
print(yo.calc_connections(yo.group1, yo.group1))

[[9, 10], [9, 13], [9, 14], [10, 11], [10, 12], [13, 15], [14, 15]]
7


In [109]:
yo.group1

[8, 9, 10, 11, 12, 13, 14, 15]

In [105]:
yo.chromosome

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]